In [1]:
# Goals: to find 800 repository and get ML to classify its language

# ACQUIRE

In [15]:
# import
import requests
import json
from bs4 import BeautifulSoup

In [16]:
# set up API request headers
from env import github_token, github_username

headers = {
    "Authorization": f"token {github_token}",
    "User-Agent": github_username
}


In [25]:
# make API requests to get repository names
language = "python"
api_url = f"https://api.github.com/search/repositories?q=language:{language}&sort=stars&order=desc&per_page=100"
response = requests.get(api_url, headers=headers)

if response.status_code == 200:
    repositories = response.json()["items"]
else:
    print(f"Failed to fetch repository list. Status code: {response.status_code}")
    exit()


In [26]:

url_to_call = "https://github.com/search?q=breast%20cancer&type=repositories"


In [27]:
breast_cancer_request = requests.get(url_to_call)

In [28]:
breast_cancer_request

<Response [200]>

In [29]:
breast_cancer_request.json()['payload']['results']

[{'id': '160983886',
  'archived': False,
  'color': '#DA5B0B',
  'followers': 795,
  'has_funding_file': False,
  'hl_name': 'nyukat/<em>breast</em>_<em>cancer</em>_classifier',
  'hl_trunc_description': "Deep Neural Networks Improve Radiologists' Performance in <em>Breast</em> <em>Cancer</em> Screening",
  'language': 'Jupyter Notebook',
  'mirror': False,
  'owned_by_organization': True,
  'public': True,
  'repo': {'repository': {'id': 160983886,
    'name': 'breast_cancer_classifier',
    'owner_id': 38415695,
    'owner_login': 'nyukat',
    'updated_at': '2023-10-03T21:42:39.109Z',
    'has_issues': True}},
  'sponsorable': False,
  'topics': ['deep-learning',
   'neural-network',
   'tensorflow',
   'pytorch',
   'medical-imaging',
   'classification',
   'pretrained-models',
   'breast-cancer',
   'medical-image-analysis',
   'breast-cancer-diagnosis'],
  'type': 'Public',
  'help_wanted_issues_count': 0,
  'good_first_issue_issues_count': 0,
  'starred_by_current_user': False

Goal: add all these results to a list, then find a way to go through and request another 10 results with always adding them to a list. clean up emphasis text from entries

In [75]:
def get_repository_details(repo_full_name):
    repo_url = f"https://api.github.com/repos/{repo_full_name}"
    readme_url = f"https://api.github.com/repos/{repo_full_name}/readme"
    repo_response = requests.get(repo_url, headers=headers)
    readme_response = requests.get(readme_url, headers=headers)
    if repo_response.status_code == 200 and readme_response.status_code == 200:
        repo_data = repo_response.json()
        readme_data = readme_response.json()
        language = repo_data["language"]
        readme_contents = requests.get(readme_data["download_url"]).text
        return {
            "repo": "/" + repo_full_name,
            "language": language,
            "readme_contents": readme_contents
        }
    else:
        return None









In [76]:
# Your GitHub API token and username
from env import github_token, github_username

# Define the headers with your token and username
headers = {
    "Authorization": f"token {github_token}",
    "User-Agent": github_username
}

# Define a function to get repositories based on a query and page
def get_repositories(query, sort="stars", order="desc", page=1):
    base_url = "https://api.github.com/search/repositories"
    params = {
        "q": query,
        "sort": sort,
        "order": order,
        "per_page": 10,  # You can get up to 100 results per page.
        "page": page  # Specify the page number
    }
    response = requests.get(base_url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Search for breast cancer related repositories and handle pagination
query = "breast cancer"
page = 1

while page <=10:
    response_json = get_repositories(query, page=page)
    if response_json is None:
        print(f"Failed to fetch page {page} of repositories.")
        break

    # Extract and print repository names
    for repo in response_json["items"]:
        print(repo["full_name"])

    # Check if there are more pages
    if "Link" in response.headers:
        next_link = response.headers["Link"]
        if 'rel="next"' not in next_link:
            break
    else:
        break
    page += 1



nyukat/breast_cancer_classifier
lishen/end2end-all-conv
ImagingLab/ICIAR2018
Jean-njoroge/Breast-cancer-risk-prediction
abhinavsagar/breast-cancer-classification
bupt-ai-cz/BCI
nyukat/BIRADS_classifier
nyukat/GMIC
srimani-programmer/Breast-Cancer-Predictor
ResearchKit/ShareTheJourney
Swarbricklab-code/BrCa_cell_atlas
ecobost/cnn4brca
Adamouization/Breast-Cancer-Detection-Mammogram-Deep-Learning
yala/Mirai
akshaybahadur21/BreastCancer_Classification
Dipeshtamboli/Image-Classification-and-Localization-using-Multiple-Instance-Learning
viritaromero/Breast-Cancer-Detection
AFAgarap/wisconsin-breast-cancer
cheersyouran/cancer-detector
bupt-ai-cz/BALNMP
cystanford/breast_cancer_data
mrdvince/breast_cancer_detection
dangnh0611/kaggle_rsna_breast_cancer
gscdit/Breast-Cancer-Detection
microsoft/Machine-Learning-Patient-Risk-Analyzer-SA
sayakpaul/Breast-Cancer-Detection-using-Deep-Learning
ezgisubasi/breast-cancer-gene-expression
Elhamkesh/Breast-Cancer-Scikitlearn
Anki0909/BreakHist-Dataset-Imag

In [60]:
# Save data to JSON file 
with open("breast_cancer_repositories.json", "w") as json_file:
    json.dump(repos, json_file, indent=4)

In [77]:
repo_list = get_repositories("breast cancer")
all_details = []
for repo in repo_list:
    details = get_repository_details(repo)
    if details:
        all_details.append(details)
# Save to github_data.json
with open("github_data.json", "w") as f:
    json.dump(all_details, f, indent=4)